# **Introductory example**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Bxn0IUVMqUqkWbMwcYn_Ia670kcdkVJi?usp=sharing)

This notebook illustrates the process to estimate parameters for three different spatial ground-motion correlation models proposed in following study

> Bodenmann, L., Baker, J. W., and Stojadinović, B. (2023): Accounting for path and site effects in spatial ground-motion correlation models using Bayesian inference, Nat. Hazards Earth Syst. Sci. Discuss. [preprint], doi: [10.5194/nhess-2022-267](https://doi.org/10.5194/nhess-2022-267).

This notebook and further supporting codes are published on Zenodo and GitHub. doi: [10.5281/zenodo.7124213](https://doi.org/10.5281/zenodo.7124213)

The notebook is structured as following:
- Packages description and import
- Data description and import
- Model specification
- Parameter estimation via MCMC
- Post-Processing

## Package description and import

We use pandas and numpy for working with dataframes and basic matrix calculus. 

In [1]:
import pandas as pd
print('pandas v' + str(pd.__version__))
import numpy as np
print('numpy v' + str(np.__version__))

pandas v1.5.3
numpy v1.22.4


Numpyro (https://num.pyro.ai/en/latest/getting_started.html) is a probabilistic programming library that supports different MCMC algorithms for high-performance inference. It uses JAX (https://github.com/google/jax) for automatic differentiation. 

In [2]:
# Install numpyro via pip
# Uncomment the following for not suppressing cell outputs
%%capture
!pip install numpyro

In [3]:
import numpyro as numpyro
print('numpyro v' + str(numpyro.__version__))
import jax as jax
print('jax v' + str(jax.__version__))

numpyro v0.11.0
jax v0.4.10


In [4]:
# Change JAX default from float32 to float64
jax.config.update("jax_enable_x64", True)

# Import some functions directly for convenience
import jax.numpy as jnp
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS

## Data description and import

The reader is referred to the main study for a complete description of the employed data set. Here we import the data used to estimate the spatial correlation models for within-event residuals of $Sa(1s)$. The variables recid and eqid relate to the record id and earthquake id from the PEER NGAwest2 flatfile (www.apps.peer.berkeley.edu/ngawest2/databases/). 

We computed the epicentral distance and the epicentral azimuth using the geo-coordinates of the earthquake epicenter and the stations that recorded the corresponding earthquake. Note that the epicentral azimuth is in radians and not degrees, while the epicentral distance is in km. 

In [5]:
# Load data from github repo
url = 'https://raw.githubusercontent.com/bodlukas/ground-motion-correlation-bayes/main/data/ngawest2/data_ngawest2_T100.csv'
df = pd.read_csv(url)

In [6]:
df

,recid,eqid,magnitude,region,station_longitude,station_latitude,epi_dist,epi_azimuth,vs30,scaled_deltaW
0,50,30,6.61,CALIFORNIA,-118.38700,33.80100,70.912551,3.111553,280.56,-0.236971
1,51,30,6.61,CALIFORNIA,-116.67400,33.55560,188.002128,2.111231,360.45,0.662453
2,52,30,6.61,CALIFORNIA,-119.02100,35.37000,117.314893,-0.493050,241.41,-1.388628
3,53,30,6.61,CALIFORNIA,-116.37000,33.26000,229.718498,2.167012,338.54,0.456357
4,54,30,6.61,CALIFORNIA,-119.35000,35.16000,117.383774,-0.817753,385.69,-0.859059
...,...,...,...,...,...,...,...,...,...,...
13337,19403,1264,3.43,CALIFORNIA,-117.09800,34.10600,34.658112,1.671885,698.72,0.078506
13338,19404,1264,3.43,CALIFORNIA,-117.67822,34.38203,33.074993,-0.611695,586.00,1.083588
13339,19406,1264,3.43,CALIFORNIA,-118.11783,34.48364,70.755517,-0.995137,405.00,-0.142001
13340,19407,1264,3.43,CALIFORNIA,-117.32960,34.56065,48.662060,0.271310,374.50,-0.393741


The data $\mathcal{D}_k=(\mathbf{X}_k,\mathbf{z}_k)$ from event $k$ consists of the input matrix $\mathbf{X}_k$ and the corresponding output vector with scaled within-event residuals $\mathbf{z}_k$. 

Each row of $\mathbf{X}_k$ stores the epicentral distance $r_{i}$, the epicentral azimuth $\theta_{i}$ and the corresponding soil condition proxy $v_{s30,i}$ for station $i$ that recorded event $k$. 

In [7]:
# Example for one event:
X = df[df.eqid==30][['epi_dist', 'epi_azimuth', 'vs30']].values
z = df[df.eqid==30]['scaled_deltaW'].values

## Model specification

All the functions defined below are also available in the script `models_jax.py`. They are re-defined here to make this a stand-alone example that can be run without any local python installation (for example on Google Colab). 

### Functions to compute distance metrics from inputs

We use polar coordinates $(r_i,\theta_i)$ to express the location of the seismic network stations with respect to the epicenter. 

Thus, the Euclidean distance between sites is computed as
$$
d_{\mathrm{E},ij} = \sqrt{r_i^2+r_j^2-2r_ir_j\cos(|\theta_i - \theta_j|)}
$$

In [8]:
def getEucDistanceFromPolar(X):
    sq_dist = (jnp.square(jnp.reshape(X[:,0],[-1,1])) + 
               jnp.square(jnp.reshape(X[:,0],[1,-1])) - 
        2 * (jnp.reshape(X[:,0],[-1,1]) * jnp.reshape(X[:,0],[1,-1])) * 
        jnp.cos(jnp.abs(jnp.reshape(X[:,1],[1,-1]) - jnp.reshape(X[:,1],[-1,1]) )) )
    sq_dist = jnp.clip(sq_dist, 0, np.inf)
    dist = jnp.sqrt(sq_dist)
    dist = dist.at[jnp.diag_indices_from(dist)].set(0.0)
    return dist

The angular distance between sites is computed as
$$
d_{\mathrm{A},ij} = \arccos(\cos(\theta_i - \theta_j))
$$

In [9]:
def getAngDistanceFromPolar(X):
    cos_angle = jnp.cos( jnp.abs(jnp.reshape(X[:,1],[1,-1]) - 
                                jnp.reshape(X[:,1],[-1,1]) ))
    dist =  jnp.arccos(jnp.clip(cos_angle, -1, 1))
    return dist

The soil dissimilarity between sites is computed as
$$
d_{\mathrm{S},ij} = |v_{s30,i}-v_{s30,j}| 
$$

In [10]:
def getSoilDissimilarity(X):
    sq_dist = jnp.square(jnp.reshape(X[:,2], [-1,1]) - jnp.reshape(X[:,2], [1,-1]))
    sq_dist = jnp.clip(sq_dist, 0, np.inf)
    dist = jnp.sqrt(sq_dist)
    return dist

### Functional forms and prior distributions

Isotropic model based on Euclidean distance (Model E)
$$
\rho_{\mathrm{E}}(d_{\mathrm{E}}; \boldsymbol{\psi}_{\mathrm{E}}) = \exp(- (d_{\mathrm{E}}/\ell_{\mathrm{E}})^{\gamma_{\mathrm{E}}})
$$

To enable the paralell estimation of parameters $\ell$ and $\gamma$, we define the function for transformed lengthscale $\ell_{\mathrm{E}t}=(\ell_{\mathrm{E}})^{\gamma_{\mathrm{E}}}$, which is called `LEt` afterwards. 

The model itself includes the sampling distributions for the priors. As described in the main study, several priors are defined on scaled versions of the parameters, for example: $p(\gamma/2) = \mathrm{Beta}(2,2)$. Thus we define the sampling distribution for `gamma2` and then compute the transformed parameter `gammaE`.


In [11]:
# Deterministic correlation functions

def rhoE(X, LEt, gammaE, nugget=1e-6):
    distE = getEucDistanceFromPolar(X)
    K = jnp.exp(-1.0 * jnp.multiply(jnp.power(distE, gammaE), 1.0/LEt))

    K = K.at[jnp.diag_indices(X.shape[0])].add(nugget)
    return K

# Probabilistic correlation model

def modelE(X, eqids, z):
    # Define Prior Distributions
    LE = numpyro.sample("LE", dist.InverseGamma(concentration=2, rate=30))
    gamma2 = numpyro.sample("gamma2", dist.Beta(2,2))

    # Compute transformed parameters
    gammaE = numpyro.deterministic("gammaE", 2.0*gamma2)
    LEt = numpyro.deterministic("LEt", jnp.power(LE, gammaE))

    # Specify observational model for each event
    z = [numpyro.sample("z_{}".format(eqid), 
              dist.MultivariateNormal(0, rhoE(X[i], LEt, gammaE)), 
              obs = z[i]) for i,eqid in enumerate(eqids)]

Accounting for path effects using angular distance (Model EA)
$$
\rho_{\mathrm{EA}}(d_{\mathrm{E}}, d_{\mathrm{A}}; \boldsymbol{\psi}_{\mathrm{EA}}) = \rho_{\mathrm{E}}(d_{\mathrm{E}}; \boldsymbol{\psi}_{\mathrm{E}}) \cdot \rho_{\mathrm{A}}(d_{\mathrm{A}}; \ell_{\mathrm{A}})
$$
where 
$$
\rho_{\mathrm{A}}(d_{\mathrm{A}}; \ell_{\mathrm{A}}) = (1 + d_{\mathrm{A}}/\ell_{\mathrm{A}})(1 - d_{\mathrm{A}}/180)^{180/\ell_{\mathrm{A}}}
$$

As mentioned in the main study the prior is defined on a transformed version of the angular lengthscale, $\ell_{\mathrm{A}}$, to respect its domain $(0°,45°)$. The transformed version is specified as `LAt` in the code below.

In [12]:
def rhoEA(X, LEt, gammaE, LA, nugget=1e-6):

    distE = getEucDistanceFromPolar(X)
    KE = jnp.exp(-1.0 * jnp.multiply(jnp.power(distE, gammaE), 1.0/LEt))

    distA = getAngDistanceFromPolar(X)
    distAdeg = jnp.multiply(distA, 180.0/np.pi)
    KA = ((1 + jnp.multiply(distAdeg, 1.0/LA)) * 
          jnp.power(1 - jnp.multiply(distAdeg, 1.0/180), 180/LA))
    
    K = KE * KA
    K = K.at[jnp.diag_indices(X.shape[0])].add(nugget)
    return K

def modelEA(X, eqids, z):
    # Define Prior Distributions
    LE = numpyro.sample("LE", dist.InverseGamma(concentration=2, rate=30))
    gamma2 = numpyro.sample("gamma2", dist.Beta(2,2))
    LAt = numpyro.sample("LAt", dist.Gamma(2, 0.25))

    # Compute transformed parameters
    gammaE = numpyro.deterministic("gammaE", 2.0*gamma2)
    LEt = numpyro.deterministic("LEt", jnp.power(LE, gammaE))
    LA = numpyro.deterministic("LA", 180/(4.0 + LAt))

    # Specify observational model for each event
    z = [numpyro.sample("z_{}".format(eqid), 
              dist.MultivariateNormal(0, rhoEA(X[i], LEt, gammaE, LA)), 
              obs = z[i]) for i,eqid in enumerate(eqids)]

Accounting for site effects using soil dissimilarity (Model EAS)
$$
\rho_{\mathrm{EAS}}(d_{\mathrm{E}}, d_{\mathrm{A}}, , d_{\mathrm{S}}; \boldsymbol{\psi}_{\mathrm{EAS}}) = \rho_{\mathrm{E}}(d_{\mathrm{E}}; \boldsymbol{\psi}_{\mathrm{E}}) \cdot (w \rho_{\mathrm{A}}(d_{\mathrm{A}}; \ell_{\mathrm{A}}) + (1-w) \rho_{\mathrm{S}}(d_{\mathrm{S}}; \ell_{\mathrm{S}}))
$$
where 
$$
\rho_{\mathrm{S}}(d_{\mathrm{S}}; \ell_{\mathrm{S}}) = \exp(-d_{\mathrm{S}}/\ell_{\mathrm{S}})
$$

In [13]:
def rhoEAS(X, LEt, gammaE, LA, LS, w, nugget=1e-6):
    
    distE = getEucDistanceFromPolar(X)
    KE = jnp.exp(-1.0 * jnp.multiply(jnp.power(distE, gammaE), 1.0/LEt))
    
    distA = getAngDistanceFromPolar(X)
    distAdeg = jnp.multiply(distA, 180.0/np.pi)
    KA = ((1 + jnp.multiply(distAdeg, 1.0/LA)) * 
          jnp.power(1 - jnp.multiply(distAdeg, 1.0/180), 180/LA))
    
    distS = getSoilDissimilarity(X)
    KS = jnp.exp(-1.0 * jnp.multiply(distS, 1.0/LS))
    
    K = KE * (w * KA + (1-w) * KS)
    K = K.at[jnp.diag_indices(X.shape[0])].add(nugget)
    return K

def modelEAS(X, eqids, z):
    # Define Prior Distributions
    LE = numpyro.sample("LE", dist.InverseGamma(concentration=2, rate=30))
    gamma2 = numpyro.sample("gamma2", dist.Beta(2,2))
    LAt = numpyro.sample("LAt", dist.Gamma(2, 0.25))
    LS = numpyro.sample("LS", dist.InverseGamma(2, 100))
    w = numpyro.sample("w", dist.Beta(2,2))

    # Compute transformed parameters
    gammaE = numpyro.deterministic("gammaE", 2.0*gamma2)
    LEt = numpyro.deterministic("LEt", jnp.power(LE, gammaE))
    LA = numpyro.deterministic("LA", 180/(4.0 + LAt))

    # Specify observational model for each event
    z = [numpyro.sample("z_{}".format(eqid), 
              dist.MultivariateNormal(0, rhoEAS(X[i], LEt, gammaE, LA, LS, w)), 
              obs = z[i]) for i,eqid in enumerate(eqids)]

## Parameter estimation via MCMC

In the main study we performed parameter estimation using four parallell MCMC chains on a high-performance computer and using following training sets $\mathcal{D}_{train}$: 

- Data $\mathcal{D}_k \subset \mathcal{D}_{tot}$ from each event $k$ to estimate parameters of event-specific isotropic models E
- Data $\mathcal{D}_{tot}$ from all 128 events to estimate parameters of pooled models E, EA and EAS

To reproduce the results from the main study please check out the provided scripts in the repository.

Here the methodology is illustrated for a training set pooled from two events $\mathcal{D}_{train}=\{\mathcal{D}_{train,1}, \mathcal{D}_{train,2} \}$ and a reduced number of two MCMC chains. 

In [14]:
numpyro.set_platform("cpu")
# Number of CPUs -> To sample several MCMC chains in parallell
numpyro.set_host_device_count(2)

In [15]:
# Pool data from two events
eqids = [30, 76] # For event-specific models use only one eqid value (e.g., [30])
Xtot = []; ztot = [] 
for eqid in eqids:
  group = df[df.eqid==eqid].copy()
  Xtot.append(group[['epi_dist', 'epi_azimuth', 'vs30']].values)
  ztot.append(group['scaled_deltaW'].values)

train_data = dict(X=Xtot, eqids=eqids, z=ztot)

Here wo only store the MCMC samples for each model in `results`. 

We could also store the results as an `inferenceData` object from `arviz`, which enables convenient plotting to check the convergence of MCMC chains. This object can also be stored locally as a `.netcdf` file. Arviz (https://arviz-devs.github.io/arviz/index.html) is a library for post-processing and illustrating results from Bayesian computations and MCMC in particular.

In [16]:
# Choose separate seeds for each model
seeds = [0, 91, 31]
results = []
models = [modelE, modelEA, modelEAS]

for m, seed in zip(models, seeds):
  # Specify model and MCMC parameters
  model = MCMC(NUTS(m), num_warmup=1000, num_samples=1000, num_chains=2)
  # Run MCMC
  model.run(jax.random.PRNGKey(seed), **train_data)
  # Store samples
  results.append(model.get_samples())
  # For post-processing with arviz use
  #idata = az.from_numpyro(model)
  #results.append(idata)


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

## Post-Processing

Here we will focus on the main results that are required to apply the model for regional seismic risk simulations (as illustrated in a separate notebook) and for the evaluation of predictive accuracy for training data and testing data.

In a first step, we extract the parameter sets $\boldsymbol{\psi}_{\mathrm{M},r}$ sampled from the posterior $p(\boldsymbol{\psi}_{\mathrm{M}}|\mathcal{D})$. Above we specified to generate 1000 samples per chain, and we used two chains. Thus, we have $n_r = 2000$ sampled parameter sets, which we store in three separate dataframes. Similar dataframes are available in the github repository for the models estimated from the pooled dataset that compromises all 128 events (instead of only the two events used above). 

In [17]:
# Parameter sets
vars = [
    ["LE", "gammaE"],
    ["LE", "gammaE", "LA"],
    ["LE", "gammaE", "LA", "LS", "w"]
]
df_param_samples = [pd.DataFrame(res)[var] 
                    for res, var in zip(results, vars)]

In [18]:
n_r = len(df_param_samples[0])
print('Example for model E')
df_param_samples[0]

Example for model E


,LE,gammaE
0,6.753899,0.547461
1,6.838889,0.475283
2,13.779938,0.607046
3,8.476057,0.473296
4,11.589176,0.475111
...,...,...
1995,5.247061,0.388825
1996,11.761814,0.350561
1997,8.258492,0.382134
1998,15.915865,0.504803


In a second step, we show how we can evaluate the predictive accuracy of the estimated models for some test data $\mathcal{D}_{test}$ using the Logarithmic Posterior Predictive Density (LPPD) 

$$
\mathrm{LPPD_{\mathrm{M}}}(\mathcal{D}_{test}|\mathcal{D}_{train}) \approx 
\ln \frac{1}{n_r} \sum_{r=1}^{n_r} p(\mathcal{D}_{test}|\boldsymbol{\psi}_{\mathrm{M},r})
$$

First, we show how to compute the log-likelihood $\ \ln p(\mathcal{D}_{test}|\boldsymbol{\psi}_{\mathrm{M}r}) \ $ conditional on each sampled parameter set $\boldsymbol{\psi}_{\mathrm{M},r}$. As a test set we consider data from two different events $\mathcal{D}_{test} = \{\mathcal{D}_{test,1}, \mathcal{D}_{test,1}\}$.

In [19]:
eqids = [114, 118]
Xtot = []; ztot = [] 
for eqid in eqids:
  group = df[df.eqid==eqid].copy()
  Xtot.append(group[['epi_dist', 'epi_azimuth', 'vs30']].values)
  ztot.append(group['scaled_deltaW'].values)

test_data = dict(X=Xtot, eqids=eqids, z=ztot)

Note that to analyze the in-sample performance of the models use `train_data` below (instead of `test_data`).

In [20]:
from numpyro.infer import log_likelihood
loglik = [pd.DataFrame(log_likelihood(m, res, **test_data))
            for res, m in zip(results, models)]

The function `log_likelihood` returns the conditional log-likelihood separately for the data from both events in the test data set.

In [21]:
print('Example for Model E')
loglik[0]

Example for Model E


,z_114,z_118
0,-72.265052,-101.592585
1,-72.504881,-100.609593
2,-69.959467,-105.722213
3,-71.965789,-100.673999
4,-71.228714,-101.299597
...,...,...
1995,-73.409632,-100.185294
1996,-72.095266,-100.389828
1997,-72.509354,-100.048995
1998,-70.417206,-103.314147


To compute the log-likelihood of the pooled test set (here we pooled data from two events), we sum the values in each row:
$$
\ln p(\mathcal{D}_{test}|\boldsymbol{\psi}_{\mathrm{M},r}) = 
\sum_{k=1}^{2} \ln p(\mathcal{D}_{test,k}|\boldsymbol{\psi}_{\mathrm{M},r})
$$

In [22]:
loglik_tot = [np.sum(LL.values, axis=1) for LL in loglik]

Then we compute the LPPD by numerically integrating out the parameter uncertainty, where we use the `logsumexp` function.

$$
\mathrm{LPPD_{\mathrm{M}}}(\mathcal{D}_{test}|\mathcal{D}_{train}) \approx 
\ln \frac{1}{n_r} \sum_{r=1}^{n_r} p(\mathcal{D}_{test}|\boldsymbol{\psi}_{\mathrm{M},r}) = - \ln {n_r} + \ln{\left(\sum_{r=1}^{n_r} \exp(\ln{p(\mathcal{D}_{test}|\boldsymbol{\psi}_{\mathrm{M},r}))}\right)}
$$

In [23]:
from scipy.special import logsumexp
LPPD = [-np.log(n_r) + logsumexp(LL) for LL in loglik_tot]
print('Example for model E')
LPPD[0]

Example for model E


-173.3297567761556

In the main study we often compared the LPPD to the LPPD of the independent model (assuming no correlation between any within-event residual). This is computed below.

In [24]:
# Array with observations from both events in the test data set.
ztot = np.hstack(test_data['z'])
LPPD_ind = (-0.5 * (len(ztot)*np.log(2*np.pi) 
                + np.sum(ztot**2)))
LPPD_ind

-198.31461826512654

The relative difference between the test LPPD of the different correlation models to the independent model is then computed as follows.

In [25]:
rel_diff = [(1-lppd/LPPD_ind)*100 for lppd in LPPD]
print('Example for model E')
print('Rel. difference in LPPD [%]')
rel_diff[0]

Example for model E
Rel. difference in LPPD [%]


12.59859797908025